In [73]:
import h2o
from h2o.automl import H2OAutoML
h2o.init(nthreads = -1, max_mem_size = '2g', ip = "127.0.0.1", port = 54321)

Checking whether there is an H2O instance running at http://127.0.0.1:54321 . connected.


H2O_cluster_uptime:,1 hour 33 mins
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_frikk_uz1u05
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.063 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [74]:
import pandas as pd
import os
import numpy as np

In [75]:
folder_train = '../features_train_in_use'

features_train = pd.read_csv(os.path.join(folder_train, os.listdir(folder_train)[0]))
for i in range(1,len(os.listdir(folder_train))):
    f = os.path.join(folder_train,os.listdir(folder_train)[i])
    if os.path.isfile(f):
        features_train = pd.concat([features_train, pd.read_csv(f)], axis=1)
print(features_train)

       average_income_per_grunnkrets  C_1.1.1.0  C_1.1.2.0  C_1.1.3.0  \
0                           561700.0          1          0          0   
1                           555720.0          1          0          0   
2                           495900.0          1          0          0   
3                           521240.0          1          0          0   
4                           585360.0          1          0          0   
...                              ...        ...        ...        ...   
12854                       505640.0          0          0          0   
12855                       396640.0          0          0          0   
12856                       549120.0          0          0          0   
12857                       498660.0          0          0          0   
12858                       579780.0          0          0          0   

       C_1.1.4.0  C_1.1.5.0  C_1.1.6.1  C_1.1.6.2  C_1.1.6.3  C_1.1.6.4  ...  \
0              0          0          0     

In [109]:
folder_test = '../features_test_in_use'

features_test = pd.read_csv(os.path.join(folder_test, os.listdir(folder_test)[0]))
for i in range(1,len(os.listdir(folder_test))):
    f = os.path.join(folder_test,os.listdir(folder_test)[i])
    if os.path.isfile(f):
        features_test = pd.concat([features_test, pd.read_csv(f)], axis=1)
print(features_test)

In [77]:
target_train = pd.read_csv("../data/stores_train.csv")
target = "revenue"
target_train = target_train[target].values

#log transform
target_train = np.log1p(target_train)
features_train[target] = target_train

In [120]:
train_df = h2o.H2OFrame(features_train)
test_df = h2o.H2OFrame(features_test)
x = test_df.columns

test_df.describe()
train_df.describe()


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Rows:8578
Cols:221

,average_income_per_grunnkrets,C_1.1.1.0,C_1.1.2.0,C_1.1.3.0,C_1.1.4.0,C_1.1.5.0,C_1.1.6.1,C_1.1.6.2,C_1.1.6.3,C_1.1.6.4,C_1.1.7.0,C_1.1.9.0,C_1.2.2.0,C_1.2.3.0,C_1.2.4.0,C_1.3.2.0,C_2.1.1.0,C_2.1.2.0,C_2.1.3.0,C_2.1.4.0,C_2.1.6.0,C_2.2.1.0,C_2.3.1.0,C_2.3.2.0,C_2.3.3.0,C_2.3.4.0,C_2.4.1.0,C_2.4.2.0,C_2.4.5.0,C_2.4.6.0,C_2.5.2.0,C_2.6.1.0,C_2.6.2.0,C_2.6.3.2,C_2.6.5.0,C_2.6.6.1,C_2.7.1.0,C_2.7.2.0,C_2.7.3.0,C_2.7.5.0,C_2.7.6.0,C_2.8.1.0,C_2.8.10.0,C_2.8.2.0,C_2.8.3.0,C_2.8.4.0,C_2.8.5.0,C_2.8.6.0,C_2.9.1.0,C_2.9.2.0,C_2.9.3.0,C_2.9.4.0,C_2.9.5.0,C_2.9.7.0,C_2.9.8.0,C_2.9.9.0,C_3.2.1.0,C_3.2.2.0,C_3.2.4.0,C_3.3.3.0,C_3.3.4.0,C_3.3.5.0,C_3.3.6.0,C_3.4.2.0,C_3.4.3.0,C_3.6.1.0,A_AKERSGATA,A_BARSTÃ˜LVEIEN,A_BEDDINGEN,A_BOGSTADVEIEN,A_BRAGERNES TORG,A_BROCHS GATE,A_BRODTKORBSGATE,A_BRUGATA,A_BYGDÃ˜Y ALLÃ‰,A_BÃ˜GATA,A_C.M. HAVIGS GATE,A_CECILIE THORESENS VEI,A_DIKEVEIEN,A_DRAMMENSVEIEN,A_DRONNINGENS GATE,A_EIDSGATA,A_EKEBERGVEIEN,A_FALKENBORGVEGEN,A_FJORDGATA,A_FRIDTJOF NANSENS GATE,A_FROGNERVEIEN,A_GAMLE DRAMMENSVEI,A_GAMLE RINGERIKSVEI,A_GAMLE STOKKAVEI,A_GATE,A_GLADBAKKVEGEN,A_GRÃ˜NLAND,A_GRÃ˜NNEGATA,A_GULDLISTEN,A_HAAKON VII'S GATE,A_HAFSTADVEGEN,A_HAMNEGATA,A_HARALDSGATA,A_HARDANGERVEGEN,A_HAVNEGATA,A_HEGDEHAUGSVEIEN,A_HENRIK IBSENS GATE,A_HENRIK WERGELANDS GATE,A_HOLTERVEIEN,A_HOVEDGATA,A_INDUSTRIGATA,A_INDUSTRIVEIEN,A_JERNBANEGATA,A_JERNBANESVINGEN,A_JERNBANETORGET,A_JERNBANEVEIEN,A_JON TORBERGSSONS VEI,A_KAIGATA,A_KARL JOHANS GATE,A_KARLSÃ˜YVEGEN,A_KIRKEGATA,A_KIRKEGATEN,A_KIRKEVEIEN,A_KIRSTEN FLAGSTADS VEI,A_KJÃ˜PMANNSGATA,A_KLOKKERGÃ…RDVEIEN,A_KONG OSCARS GATE,A_KONGENS GATE,A_KVERNHUSMYRANE,A_KYRKJEVEGEN,A_LAGUNEVEIEN,A_LANGELANDSVEGEN,A_LANGGATA,A_LANGSÃ†VEIEN,A_LIAMYRANE,A_LIEGATA,A_LILLEAKERVEIEN,A_LODDEFJORDVEIEN,A_LONGHAMMARVEGEN,A_LÃ˜KKEVEIEN,A_MADLAKROSSEN,A_MARIDALSVEIEN,A_MARKEDSGATA,A_MARKENS GATE,A_MARKVEIEN,A_MUNKEDAMSVEIEN,A_MUNKEGATA,A_MÃ˜LLERGATA,A_NEDRE LANGGATE,A_NEDRE STORGATE,A_NIELS JUELS GATE,A_NORDRE GATE,A_NYGAARDSGATA,A_OLAV TRYGGVASONS GATE,A_ORKDALSVEIEN,A_OSCARS GATE,A_PARKVEIEN,A_PILESTREDET,A_PRINSENS GATE,A_RINGVEGEN,A_ROALD AMUNDSENS GATE,A_RUSELÃ˜KKVEIEN,A_RÃ…DHUSGATA,A_RÃ…DHUSVEIEN,A_RÃ…DMANN HALMRASTS VEI,A_SANDAKERVEIEN,A_SANDVIKSVEIEN,A_SENTRUMSVEGEN,A_SENTRUMSVEIEN,A_SJÃ˜FARTSGATA,A_SJÃ˜GATA,A_SKAGEN,A_SKIPPERGATA,A_SKOLEGATA,A_SKOVVEIEN,A_SOGNSVEIEN,A_ST. MARIE GATE,A_STASJONSGATA,A_STASJONSVEGEN,A_STASJONSVEIEN,A_STOKMOVEGEN,A_STORE ELVEGATE,A_STORGATA,A_STORGATA,A_STORGATEN,A_STORTINGSGATA,A_STOVNER SENTER,A_STRANDGATA,A_STRANDGATEN,A_STRANDVEGEN,A_STRANDVEIEN,A_STRÃ˜MGATEN,A_STRÃ˜MSVEIEN,A_STÃ˜PERIVEIEN,A_SÃ˜NDRE GATE,A_SÃ˜RKEDALSVEIEN,A_TASTATUNET,A_THERESES GATE,A_THORVALD MEYERS GATE,A_TOLLBODGATA,A_TOLLBUGATA,A_TORDENSKIOLDS GATE,A_TORGALLMENNINGEN,A_TORGET,A_TORGGATA,A_TORVET,A_TRONDHEIMSVEIEN,A_TVETENVEIEN,A_ULEFOSSVEGEN,A_ULLEVÃ…LSVEIEN,A_VANGSGATA,A_VERFTSGATA,A_VERKSTEDVEIEN,A_VITAMINVEIEN,A_VOGTS GATE,A_VÃ†KERÃ˜VEIEN,A_WALDEMAR THRANES GATE,A_Ã˜KERNVEIEN,A_Ã˜STERVÃ…G,A_Ã˜STRE ROSTEN,A_Ã˜VRE LANGGATE,distance_store_busstop,distance_to_competitor,has_mall_or_not,area_grunnkrets
type,real,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,real,int,real
mins,0.0,0.0,0.0,0.0,

Rows:12860
Cols:222

,average_income_per_grunnkrets,C_1.1.1.0,C_1.1.2.0,C_1.1.3.0,C_1.1.4.0,C_1.1.5.0,C_1.1.6.1,C_1.1.6.2,C_1.1.6.3,C_1.1.6.4,C_1.1.7.0,C_1.1.9.0,C_1.2.2.0,C_1.2.3.0,C_1.2.4.0,C_1.3.2.0,C_2.1.1.0,C_2.1.2.0,C_2.1.3.0,C_2.1.4.0,C_2.1.6.0,C_2.2.1.0,C_2.3.1.0,C_2.3.2.0,C_2.3.3.0,C_2.3.4.0,C_2.4.1.0,C_2.4.2.0,C_2.4.5.0,C_2.4.6.0,C_2.5.2.0,C_2.6.1.0,C_2.6.2.0,C_2.6.3.2,C_2.6.5.0,C_2.6.6.1,C_2.7.1.0,C_2.7.2.0,C_2.7.3.0,C_2.7.5.0,C_2.7.6.0,C_2.8.1.0,C_2.8.10.0,C_2.8.2.0,C_2.8.3.0,C_2.8.4.0,C_2.8.5.0,C_2.8.6.0,C_2.9.1.0,C_2.9.2.0,C_2.9.3.0,C_2.9.4.0,C_2.9.5.0,C_2.9.7.0,C_2.9.8.0,C_2.9.9.0,C_3.2.1.0,C_3.2.2.0,C_3.2.4.0,C_3.3.3.0,C_3.3.4.0,C_3.3.5.0,C_3.3.6.0,C_3.4.2.0,C_3.4.3.0,C_3.6.1.0,A_AKERSGATA,A_BARSTÃ˜LVEIEN,A_BEDDINGEN,A_BOGSTADVEIEN,A_BRAGERNES TORG,A_BROCHS GATE,A_BRODTKORBSGATE,A_BRUGATA,A_BYGDÃ˜Y ALLÃ‰,A_BÃ˜GATA,A_C.M. HAVIGS GATE,A_CECILIE THORESENS VEI,A_DIKEVEIEN,A_DRAMMENSVEIEN,A_DRONNINGENS GATE,A_EIDSGATA,A_EKEBERGVEIEN,A_FALKENBORGVEGEN,A_FJORDGATA,A_FRIDTJOF NANSENS GATE,A_FROGNERVEIEN,A_GAMLE DRAMMENSVEI,A_GAMLE RINGERIKSVEI,A_GAMLE STOKKAVEI,A_GATE,A_GLADBAKKVEGEN,A_GRÃ˜NLAND,A_GRÃ˜NNEGATA,A_GULDLISTEN,A_HAAKON VII'S GATE,A_HAFSTADVEGEN,A_HAMNEGATA,A_HARALDSGATA,A_HARDANGERVEGEN,A_HAVNEGATA,A_HEGDEHAUGSVEIEN,A_HENRIK IBSENS GATE,A_HENRIK WERGELANDS GATE,A_HOLTERVEIEN,A_HOVEDGATA,A_INDUSTRIGATA,A_INDUSTRIVEIEN,A_JERNBANEGATA,A_JERNBANESVINGEN,A_JERNBANETORGET,A_JERNBANEVEIEN,A_JON TORBERGSSONS VEI,A_KAIGATA,A_KARL JOHANS GATE,A_KARLSÃ˜YVEGEN,A_KIRKEGATA,A_KIRKEGATEN,A_KIRKEVEIEN,A_KIRSTEN FLAGSTADS VEI,A_KJÃ˜PMANNSGATA,A_KLOKKERGÃ…RDVEIEN,A_KONG OSCARS GATE,A_KONGENS GATE,A_KVERNHUSMYRANE,A_KYRKJEVEGEN,A_LAGUNEVEIEN,A_LANGELANDSVEGEN,A_LANGGATA,A_LANGSÃ†VEIEN,A_LIAMYRANE,A_LIEGATA,A_LILLEAKERVEIEN,A_LODDEFJORDVEIEN,A_LONGHAMMARVEGEN,A_LÃ˜KKEVEIEN,A_MADLAKROSSEN,A_MARIDALSVEIEN,A_MARKEDSGATA,A_MARKENS GATE,A_MARKVEIEN,A_MUNKEDAMSVEIEN,A_MUNKEGATA,A_MÃ˜LLERGATA,A_NEDRE LANGGATE,A_NEDRE STORGATE,A_NIELS JUELS GATE,A_NORDRE GATE,A_NYGAARDSGATA,A_OLAV TRYGGVASONS GATE,A_ORKDALSVEIEN,A_OSCARS GATE,A_PARKVEIEN,A_PILESTREDET,A_PRINSENS GATE,A_RINGVEGEN,A_ROALD AMUNDSENS GATE,A_RUSELÃ˜KKVEIEN,A_RÃ…DHUSGATA,A_RÃ…DHUSVEIEN,A_RÃ…DMANN HALMRASTS VEI,A_SANDAKERVEIEN,A_SANDVIKSVEIEN,A_SENTRUMSVEGEN,A_SENTRUMSVEIEN,A_SJÃ˜FARTSGATA,A_SJÃ˜GATA,A_SKAGEN,A_SKIPPERGATA,A_SKOLEGATA,A_SKOVVEIEN,A_SOGNSVEIEN,A_ST. MARIE GATE,A_STASJONSGATA,A_STASJONSVEGEN,A_STASJONSVEIEN,A_STOKMOVEGEN,A_STORE ELVEGATE,A_STORGATA,A_STORGATA,A_STORGATEN,A_STORTINGSGATA,A_STOVNER SENTER,A_STRANDGATA,A_STRANDGATEN,A_STRANDVEGEN,A_STRANDVEIEN,A_STRÃ˜MGATEN,A_STRÃ˜MSVEIEN,A_STÃ˜PERIVEIEN,A_SÃ˜NDRE GATE,A_SÃ˜RKEDALSVEIEN,A_TASTATUNET,A_THERESES GATE,A_THORVALD MEYERS GATE,A_TOLLBODGATA,A_TOLLBUGATA,A_TORDENSKIOLDS GATE,A_TORGALLMENNINGEN,A_TORGET,A_TORGGATA,A_TORVET,A_TRONDHEIMSVEIEN,A_TVETENVEIEN,A_ULEFOSSVEGEN,A_ULLEVÃ…LSVEIEN,A_VANGSGATA,A_VERFTSGATA,A_VERKSTEDVEIEN,A_VITAMINVEIEN,A_VOGTS GATE,A_VÃ†KERÃ˜VEIEN,A_WALDEMAR THRANES GATE,A_Ã˜KERNVEIEN,A_Ã˜STERVÃ…G,A_Ã˜STRE ROSTEN,A_Ã˜VRE LANGGATE,distance_store_busstop,distance_to_competitor,has_mall_or_not,area_grunnkrets,revenue
type,real,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,real,int,real,real
mins,0.0

In [96]:
h2o_automl = H2OAutoML(sort_metric='rmsle', max_runtime_secs=0, seed=666)

29


In [80]:
h2o_automl.train(x=x, y=target, training_frame=train_df)

AutoML progress: |
22:43:54.618: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,0.6215221,0.0085421,0.621638,0.6242085,0.6264631,0.6283799,0.6069209
mean_residual_deviance,0.6511883,0.0081563,0.6489474,0.655513,0.6502104,0.6616081,0.6396624
mse,0.6511883,0.0081563,0.6489474,0.655513,0.6502104,0.6616081,0.6396624
null_deviance,2701.602,80.54451,2668.7312,2671.8508,2774.0278,2793.371,2600.0293
r2,0.3797237,0.0144373,0.3638707,0.3655632,0.3849609,0.3872362,0.3969875
residual_deviance,1675.0245,61.37148,1697.6464,1694.5011,1705.5017,1711.5801,1565.8936
rmse,0.8069497,0.0050563,0.8055727,0.8096376,0.8063562,0.8133929,0.7997890
rmsle,0.3389875,0.0060467,0.3362414,0.3408150,0.3463432,0.3412713,0.3302668


In [81]:
#View leaderborad

lb = h2o_automl.leaderboard
lb.head()

model_id,rmsle,rmse,mse,mae,mean_residual_deviance
StackedEnsemble_AllModels_3_AutoML_4_20221107_224354,0.339138,0.807034,0.651304,0.621694,0.651304
StackedEnsemble_AllModels_4_AutoML_4_20221107_224354,0.339172,0.807046,0.651323,0.62137,0.651323
StackedEnsemble_AllModels_6_AutoML_4_20221107_224354,0.339267,0.807194,0.651562,0.622162,0.651562
StackedEnsemble_BestOfFamily_4_AutoML_4_20221107_224354,0.339464,0.808048,0.652941,0.622915,0.652941
StackedEnsemble_BestOfFamily_6_AutoML_4_20221107_224354,0.33955,0.807944,0.652774,0.623705,0.652774
StackedEnsemble_AllModels_5_AutoML_4_20221107_224354,0.339823,0.810404,0.656755,0.623341,0.656755
StackedEnsemble_AllModels_2_AutoML_4_20221107_224354,0.340016,0.809738,0.655676,0.624407,0.655676
StackedEnsemble_AllModels_1_AutoML_4_20221107_224354,0.340348,0.810569,0.657022,0.625474,0.657022
StackedEnsemble_BestOfFamily_3_AutoML_4_20221107_224354,0.340392,0.810968,0.65767,0.625647,0.65767
StackedEnsemble_BestOfFamily_5_AutoML_4_20221107_224354,0.340599,0.814112,0.662778,0.626478,0.662778


In [82]:
predictions = h2o_automl.predict(test_df)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [114]:
predictions_pd = h2o.as_list(predictions)
predictions_list = predictions_pd["predict"].tolist()
predictions_list = np.expm1(predictions_list)

       predict
0     1.600869
1     1.676974
2     1.693739
3     1.619789
4     1.543239
...        ...
8573  1.243863
8574  1.449868
8575  1.464520
8576  1.461442
8577  1.191766

[8578 rows x 1 columns]
8578
8578


In [84]:
stores_test = pd.read_csv("../data/stores_test.csv")
stores_test_id = stores_test['store_id'].values

In [122]:
predictions_df = pd.DataFrame()

predictions_df['id'] = stores_test_id
print(len(predictions_list))
predictions_df['predicted'] = predictions_list[1:]
predictions_df.to_csv("../predictions/h2o_automl2.csv", index=False)

8578


In [123]:
h2o_automl.varimp()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: Column model_id not found
  Request: POST /99/Rapids
    data: {'ast': "(tmp= py_20_sid_9730 (cols_py Key_Frame__upload_8fe0e7fafa66ec7651652f17007f4a11.hex 'model_id'))", 'session_id': '_sid_9730'}
